In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

c:\Users\ANMOL DESHRATNA\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
mnist_data,mnist_info = tfds.load(name='mnist',with_info=True,as_supervised=True)

In [3]:
mnist_train,mnist_test = mnist_data['train'],mnist_data['test']
num_validation_samples = 0.1*mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples,tf.int64) # cast() change the type of data


num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples,tf.int64)

def scale(image,label):
    image = tf.cast(image,tf.float32)
    image /= 255.
    return image,label

scaled_train_and_valid = mnist_train.map(scale)
test_data = mnist_test.map(scale) #map(*function*) allows custom transformation of a given dataset

BUFFER_SIZE = 10000

shuffled_train_valid_data = scaled_train_and_valid.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_valid_data.take(num_validation_samples)
train_data = shuffled_train_valid_data.skip(num_validation_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE) # dataset.batch(batchsize) a method that combines the consecutive elements into batches

validation_data = validation_data.batch(num_validation_samples)

test_data = test_data.batch(num_test_samples)

validation_inputs,validation_targets = next(iter(validation_data)) #iter() creates an obkect which can be iterated one element at a time
# next() loads the next batch

##MODEL


In [4]:
input_size = 784  #28 x 28 pixel size
output_size = 10


hidden_layer_size = 100

model = tf.keras.Sequential([tf.keras.layers.Flatten(input_shape=(28,28,1)), # this func transform tensor to vector
                            tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                            tf.keras.layers.Dense(output_size,activation='sigmoid'),
                            
                            ])


#CHOOSE THE Optimizer and loss funct

In [5]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 100)               10100     
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 89,610
Trainable params: 89,610
Non-trainable params: 0
_________________________________________________________________


##TRAINING

In [6]:
NUM_EPOCHS = 5

model.fit(train_data,epochs=NUM_EPOCHS,validation_data=(validation_inputs,validation_targets),verbose = 2)

Epoch 1/5
540/540 - 3s - loss: 0.3397 - accuracy: 0.9036 - val_loss: 0.1578 - val_accuracy: 0.9525 - 3s/epoch - 6ms/step
Epoch 2/5
540/540 - 2s - loss: 0.1384 - accuracy: 0.9586 - val_loss: 0.1118 - val_accuracy: 0.9652 - 2s/epoch - 4ms/step
Epoch 3/5
540/540 - 2s - loss: 0.0953 - accuracy: 0.9707 - val_loss: 0.0905 - val_accuracy: 0.9738 - 2s/epoch - 4ms/step
Epoch 4/5
540/540 - 2s - loss: 0.0731 - accuracy: 0.9774 - val_loss: 0.0860 - val_accuracy: 0.9752 - 2s/epoch - 3ms/step
Epoch 5/5
540/540 - 2s - loss: 0.0598 - accuracy: 0.9815 - val_loss: 0.0659 - val_accuracy: 0.9793 - 2s/epoch - 3ms/step


#TESTING


In [7]:
test_loss , test_accuracy = model.evaluate(test_data)
test_loss,test_accuracy

1/1 [==============================] - 0s 316ms/step - loss: 0.0881 - accuracy: 0.9754


(0.08813077211380005, 0.9753999710083008)